# Squashed Entanglement
$$
 \newcommand{\ul}[1]{\underline{#1}}
 \newcommand{\rvalp}[0]{{\ul{\alpha}}}
\newcommand{\alp}[0]{{\alpha}}
 \newcommand{\rvx}[0]{{\ul{x}}}
\newcommand{\rvy}[0]{{\ul{y}}}
$$

The purpose of this notebook is to show how to use entanglish to
calculate the squashed entanglement of a mixed state (either pure or not pure).

Consider a bipartite system
consisting of two parts labelled
by the random variables $\rvx$ and $\rvy$,
and
described by a density matrix $\rho_{\rvx, \rvy}$.
The squashed entanglement of  such a system
is defined as

$$
E_{\rvx, \rvy}(\rho_{\rvx, \rvy}) =
\frac{1}{2}
\min S(\rvx : \rvy|\rvalp)
\;.
$$
The min()---or infimum()
if one wishes to be more mathematically
precise--is
over all density matrices $\rho_{\rvx, \rvy,\rvalp}$
such that ${\rm tr}_\rvalp \; \rho_{\rvx, \rvy,\rvalp}=
\rho_{\rvx, \rvy}$ with $\rho_{\rvx, \rvy}$ held fixed.
If $\rho_{\rvx, \rvy}$ is a pure state, then
$E_{\rvx, \rvy} = S(\rvx) = S(\rvy)$.
Entanglish-Original-Ref discusses other interesting
properties of squashed entanglement

Entanglish-Original-Ref also describes the algo
used by Entanglish to calculate squashed entanglement. The
algorithm is recursive. 
The number of recursive steps 
can be chosen by the user and is called num_ab_steps (ab stands
for Arimoto-Blahut).
Another parameter of the algorithm is num_hidden_states, which is the number of possible 
$\rvalp$ values.

 
**Entanglish-Original-Ref**
* "A New  Algorithm for Calculating
Squashed Entanglement and a Python Implementation Thereof", by R.R.Tucci

First change your working directory to the entanglish directory in your computer, and add its path to the path environment variable.

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/rrtucci/PycharmProjects/Entanglish/entanglish/jupyter_notebooks
/home/rrtucci/PycharmProjects/Entanglish


In [2]:
from entanglish.SymNupState import *
from entanglish.SquashedEnt import *

## pure states (symmetrized n-up states)
Next we construct a symmetrized n-up pure state.
Then we compare the Arimoto-Blahut algo entanglement value to the known 
entanglement value, for various possible
bi-partitions of the set of row axes. As expected, they are equal.

recursion_init is a str that specifies how to
initiate the recursion for Kxy_a. There are currently 2 options for recursion_init, 'eigen' and 'eigen+'. 'eigen+' uses Dxy.num_rows^2 degrees of freedom, which is sufficient but 
may not be necessary. 'eigen' uses only Dxy.num_rows degrees of freedom.
With both 'eigen' and 'eigen+', the recursion converges instantly for pure states. 

In [3]:
num_qbits = 4
num_up = 1
dm1 = DenMat(1 << num_qbits, tuple([2]*num_qbits))
st = SymNupState(num_up, num_qbits)
st_vec = st.get_st_vec()
dm1.set_arr_from_st_vec(st_vec)

recursion_init = 'eigen+'
num_ab_steps = 5
print('recursion_init=', recursion_init)
print('num_ab_steps=', num_ab_steps)
ecase = SquashedEnt(dm1, num_ab_steps,
    recursion_init=recursion_init, verbose=True)
print('entang_023: algo value, known value\n',
      ecase.get_entang({0, 2, 3}),
      st.get_known_entang(3))
print('entang_02: algo value, known value\n',
      ecase.get_entang({0, 2}),
      st.get_known_entang(2))
print('entang_1: algo value, known value\n',
      ecase.get_entang({1}),
      st.get_known_entang(1))

recursion_init= eigen+
num_ab_steps= 5

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 2.7194799110210365e-16
--ab step= 0 , entang= 0.562335 , err= 0.000000
--ab step= 1 , entang= 0.562335 , err= 0.000000
--ab step= 2 , entang= 0.562335 , err= 0.000000
--ab step= 3 , entang= 0.562335 , err= 0.000000
--ab step= 4 , entang= 0.562335 , err= 0.000000
entang_023: algo value, known value
 0.5623351446188093 0.5623351446188083

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 2.7194799110210365e-16
--ab step= 0 , entang= 0.693147 , err= 0.000000
--ab step= 1 , entang= 0.693147 , err= 0.000000
--ab step= 2 , entang= 0.693147 , err= 0.000000
--ab step= 3 , entang= 0.693147 , err= 0.000000
--ab step= 4 , entang= 0.693147 , err= 0.000000
entang_02: algo value, known value
 0.6931471805599457 0.6931471805599453

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 2.7194799110210365e-16
--ab step= 0 , entang= 0.562335 , err= 0.000000
--ab step= 1 , entang= 0.562335 , err= 0.000000
--ab ste

## random density matrices
Next we consider 2 random density matrices (actually,
only their eigenvectors are random. Their eigenvalues are specified by the user.)
For each of those 2 density matrices, we calculate the Arimoto-Blahut algo entanglement value, for various possible
bi-partitions of the set of row axes.

In [4]:
np.random.seed(123)
dm = DenMat(8, (2, 2, 2))
evas_of_dm_list = [
    np.array([.07, .03, .25, .15, .3, .1, .06, .04])
    , np.array([.05, .05, .2, .2, .3, .1, .06, .04])
    ]

recursion_init = 'eigen+'
num_ab_steps = 100
print('recursion_init=', recursion_init)
print('num_ab_steps=', num_ab_steps)
for evas_of_dm in evas_of_dm_list:
    evas_of_dm /= np.sum(evas_of_dm)
    print('***************new dm')
    print('evas_of_dm\n', evas_of_dm)
    dm.set_arr_to_rand_den_mat(evas_of_dm)
    ecase = SquashedEnt(dm, num_ab_steps,
        recursion_init=recursion_init, verbose=True)
    print('ent_02_1=', ecase.get_entang({0, 2}))

recursion_init= eigen+
num_ab_steps= 100
***************new dm
evas_of_dm
 [0.07 0.03 0.25 0.15 0.3  0.1  0.06 0.04]

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 4.880055036277153e-16
--ab step= 0 , entang= 0.446881 , err= 26.212601
--ab step= 1 , entang= 0.169440 , err= 18.442138
--ab step= 2 , entang= 0.240223 , err= 8.346209
--ab step= 3 , entang= 0.090830 , err= 2.642048
--ab step= 4 , entang= 0.051815 , err= 0.848501
--ab step= 5 , entang= 0.041352 , err= 0.599272
--ab step= 6 , entang= 0.034656 , err= 0.515530
--ab step= 7 , entang= 0.029621 , err= 0.450331
--ab step= 8 , entang= 0.025599 , err= 0.402359
--ab step= 9 , entang= 0.022220 , err= 0.367524
--ab step= 10 , entang= 0.019274 , err= 0.344233
--ab step= 11 , entang= 0.016668 , err= 0.328257
--ab step= 12 , entang= 0.014370 , err= 0.316916
--ab step= 13 , entang= 0.012373 , err= 0.307283
--ab step= 14 , entang= 0.010676 , err= 0.298265
--ab step= 15 , entang= 0.009261 , err= 0.289201
--ab step= 16 , entang= 0.008100

--ab step= 60 , entang= 0.000688 , err= 0.069408
--ab step= 61 , entang= 0.000679 , err= 0.068246
--ab step= 62 , entang= 0.000671 , err= 0.067120
--ab step= 63 , entang= 0.000662 , err= 0.066009
--ab step= 64 , entang= 0.000655 , err= 0.064327
--ab step= 65 , entang= 0.000648 , err= 0.062313
--ab step= 66 , entang= 0.000642 , err= 0.060533
--ab step= 67 , entang= 0.000636 , err= 0.059003
--ab step= 68 , entang= 0.000630 , err= 0.057613
--ab step= 69 , entang= 0.000625 , err= 0.056295
--ab step= 70 , entang= 0.000620 , err= 0.055025
--ab step= 71 , entang= 0.000615 , err= 0.053807
--ab step= 72 , entang= 0.000611 , err= 0.052639
--ab step= 73 , entang= 0.000607 , err= 0.051523
--ab step= 74 , entang= 0.000603 , err= 0.050453
--ab step= 75 , entang= 0.000599 , err= 0.049426
--ab step= 76 , entang= 0.000596 , err= 0.048439
--ab step= 77 , entang= 0.000593 , err= 0.047488
--ab step= 78 , entang= 0.000590 , err= 0.046571
--ab step= 79 , entang= 0.000587 , err= 0.045685
--ab step= 80 , enta